In [ ]:
# term	string	Optional. Search term (e.g. "food", "restaurants"). If term isn’t included we search everything. The term keyword also accepts business names such as "Starbucks".
# location	string	Required if either latitude or longitude is not provided. Specifies the combination of "address, neighborhood, city, state or zip, optional country" to be used when searching for businesses.
# latitude	decimal	Required if location is not provided. Latitude of the location you want to search near by.
# longitude	decimal	Required if location is not provided. Latitude of the location you want to search near by.
# radius	int	Optional. Search radius in meters. If the value is too large, a AREA_TOO_LARGE error may be returned. The max value is 40000 meters (25 miles).
# categories	string	Optional. Categories to filter the search results with. See the list of supported categories. The category filter can be a list of comma delimited categories. For example, "bars,french" will filter by Bars and French. The category identifier should be used (for example "discgolf", not "Disc Golf").
# locale	string	Optional. Specify the locale to return the business information in. See the list of supported locales.
# limit	int	Optional. Number of business results to return. By default, it will return 20. Maximum is 50.
# offset	int	Optional. Offset the list of returned business results by this amount.
# sort_by	string	Optional. Sort the results by one of the these modes: best_match, rating, review_count or distance. By default it's best_match. The rating sort is not strictly sorted by the rating value, but by an adjusted rating value that takes into account the number of ratings, similar to a bayesian average. This is so a business with 1 rating of 5 stars doesn’t immediately jump to the top.
# price	string	Optional. Pricing levels to filter the search result with: 1 = $, 2 = $$, 3 = $$$, 4 = $$$$. The price filter can be a list of comma delimited pricing levels. For example, "1, 2, 3" will filter the results to show the ones that are $, $$, or $$$.
# open_now	boolean	Optional. Default to false. When set to true, only return the businesses open now. Notice that open_at and open_now cannot be used together.
# open_at	int	Optional. An integer represending the Unix time in the same timezone of the search location. If specified, it will return business open at the given time. Notice that open_at and open_now cannot be used together.
# attributes	string	Optional. Additional filters to search businesses. You can use multiple attribute filters at the same time by providing a comma separated string, like this "attribute1,attribute2". Currently, the valid values are hot_and_new and deals.

In [103]:
from boto.s3 import connect_to_region
import json
import matplotlib
import os
import pandas as pd
import requests
import sys
print (sys.version)

if 's3' in boto.config.sections():
    boto.config.remove_section('s3')
    boto.config.add_section('s3')
    boto.config.set('s3', 'use-sigv4', 'True')

2.7.12 (default, Jul 28 2016, 19:08:23) 
[GCC 4.9.2]


In [104]:
GEOJSON = 'geojson/bouroughs.geojson'

In [93]:
conn = connect_to_region(region_name='us-east-2',profile_name='default')

my_bucket = conn.get_bucket('patrickjmc')

keys = my_bucket.get_all_keys()

file_key = [k for k in keys if k.key=='yelp_api.json']
api_cred_string = file_key[0].get_contents_as_string()

yelp_credentials = json.loads(api_cred_string)

In [95]:
yelp_credentials = json.loads(api_cred_string)

In [96]:
APPID = yelp_credentials['APPID']
SECRET = yelp_credentials['SECRET']
BUSINESSSEARCHURL = 'https://api.yelp.com/v3/businesses/search'
TOKENURL = 'https://api.yelp.com/oauth2/token'

In [97]:
params = {'client_id':APPID,
         'client_secret':SECRET,
         'grant_type':'client_credentials'}

In [99]:
token_req = requests.post(TOKENURL,params)

In [100]:
TOKEN = json.loads(token_req.content)['access_token']

In [101]:
headers = {'Authorization':'Bearer '+TOKEN}

In [44]:
req = requests.get(BUSINESSSEARCHURL, params = {'term':'thai',
                                                'radius':4000,
                                                'latitude':40.7128,
                                                'longitude':-74.0059,
                                                'limit':50}, headers=headers)

In [46]:
len(json.loads(req.content)['businesses'])

50

In [51]:
[x['title'] for x in json.loads(req.content)['businesses'][0]['categories']]

[u'Thai', u'Malaysian', u'Vietnamese']

In [52]:
my_obj = json.loads(req.content)['businesses'][0]

In [55]:
my_obj

{u'categories': [{u'alias': u'thai', u'title': u'Thai'},
  {u'alias': u'malaysian', u'title': u'Malaysian'},
  {u'alias': u'vietnamese', u'title': u'Vietnamese'}],
 u'coordinates': {u'latitude': 40.7303859, u'longitude': -73.9860613},
 u'id': u'love-mamak-new-york-2',
 u'image_url': u'https://s3-media2.fl.yelpcdn.com/bphoto/bLlFKTlVuLfmF-lIDGIjZA/o.jpg',
 u'is_closed': False,
 u'location': {u'address1': u'174 2nd Ave',
  u'address2': u'',
  u'address3': u'',
  u'city': u'New York',
  u'country': u'US',
  u'state': u'NY',
  u'zip_code': u'10003'},
 u'name': u'Love Mamak',
 u'phone': u'+12122542868',
 u'price': u'$$',
 u'rating': 4.0,
 u'review_count': 1981,
 u'url': u'https://www.yelp.com/biz/love-mamak-new-york-2?adjust_creative=nzgGbAJQ1s77ZRvTEz_TaQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=nzgGbAJQ1s77ZRvTEz_TaQ'}

In [107]:
def business_extractor(bus_json):
    
    bus_dict = {}
    
    #bus_dict['category_list'] = [x['title'] for x in bus_json['categories']]
    bus_dict['id_str'] = bus_json['id']
    bus_dict['name'] = bus_json['name']
    if 'price' in bus_json.keys():
        bus_dict['price'] = len(bus_json['price'])
    else:
        bus_dict['price'] = -1
    bus_dict['rating'] = bus_json['rating']
    bus_dict['num_review'] = bus_json['review_count']
    bus_dict['lat'] = bus_json['coordinates']['latitude']
    bus_dict['lon'] = bus_json['coordinates']['longitude']
    
    #print bus_dict
    #return bus_dict
    return bus_dict

In [75]:
business_extractor(my_obj)

{'id_str': u'love-mamak-new-york-2',
 'lat': 40.7303859,
 'lon': -73.9860613,
 'name': u'Love Mamak',
 'num_review': 1981,
 'price': 2,
 'rating': 4.0}

In [76]:
pd.DataFrame(business_extractor(my_obj),index=[0])

,id_str,lat,lon,name,num_review,price,rating
0,love-mamak-new-york-2,40.730386,-73.986061,Love Mamak,1981,2,4.0


In [132]:
def make_call_df(req,rad):
    
    req = requests.get(BUSINESSSEARCHURL, params = {'term':'restaurants',
                                                'radius':rad,
                                                'latitude':40.7128,
                                                'longitude':-74.0059,
                                                'limit':50}, headers=headers)
    
    df_list = []
    
    for b in json.loads(req.content)['businesses']:
        print b
        b_df = business_extractor(b)
        df_list.append(b_df)
    
    return df_list

In [82]:
pd.DataFrame(make_call_df(req))

,id_str,lat,lon,name,num_review,price,rating
0,love-mamak-new-york-2,40.730386,-73.986061,Love Mamak,1981,2,4.0
1,thai-holic-fort-greene,40.692990,-73.971280,Thai Holic,155,2,4.5
2,pinto-new-york-2,40.733032,-74.005447,Pinto,828,2,4.0
3,thai-sliders-and-co-new-york-2,40.707713,-74.005888,Thai Sliders & Co,138,2,4.0
4,khe-yo-new-york,40.716906,-74.008587,Khe-Yo,417,3,4.0
5,lan-larb-soho-new-york,40.719994,-73.998543,Lan Larb Soho,129,2,3.5
6,zest-new-york-3,40.717834,-73.989960,Zest,529,2,4.0
7,luckyim-thai-new-york,40.704561,-74.008948,Luckyim Thai,96,1,4.0
8,somtum-der-new-york,40.725215,-73.984274,Somtum Der,410,2,4.0
9,aroy-dee-thai-kitchen-new-york,40.709682,-74.008972,Aroy Dee Thai Kitchen,242,1,3.5


In [113]:
range(0,50,5)

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45]

In [128]:
shapes = [(x,len(make_call_df(req,x))) for x in range(10)]

In [129]:
shapes

[(0, 50),
 (1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 10),
 (6, 10),
 (7, 10),
 (8, 10),
 (9, 10)]

In [130]:
test_list = []

for x in range(10):
    test_list.append(make_call_df(req,5))

test_list_df = []
for f in test_list:
    ff = pd.DataFrame(f)
    ff.sort_values('id_str',inplace=True)
    test_list_df.append(ff)
    

In [131]:
test_list_df[1]

,id_str,lat,lon,name,num_review,price,rating
9,afropolitan-insights-nyc-new-york,40.714550,-74.007140,Afropolitan Insights NYC,1,-1,1.0
8,citylisten-audio-tours-new-york,40.714550,-74.007140,CityListen Audio Tours,6,-1,4.5
1,famous-fat-daves-food-tours-new-york,40.714550,-74.007140,Famous Fat Dave's Food Tours,37,3,5.0
0,halal-food-new-york-6,40.714550,-74.007140,Halal Food,21,1,4.0
2,king-david-tacos-new-york,40.714550,-74.007140,King David Tacos,1,1,5.0
5,limo-star-ny-new-york-2,40.712784,-74.005941,Limo Star NY,9,-1,4.0
7,mr-softy-new-york-3,NaN,NaN,Mr. Softy,2,1,5.0
4,snap-dog-new-york,40.714550,-74.007140,Snap Dog,1,-1,3.0
3,the-cardinal-club-new-york,40.714550,-74.007140,The Cardinal Club,2,-1,5.0
6,west-side-highway-bike-path-new-york,40.714550,-74.007140,West Side Highway Bike Path,27,-1,4.0


In [87]:
token_req.content

'{"access_token": "QqMecfNuMuVkUSPPnPJhnxGMyWQjpTw2cZKlj4fuzvheuJFey2gP1X1dk26SfAKVWWdcsQ8fOvb1T_1NfUjywvJQDKCmJlO_4ygDAP3c9VLWfsHRX2OFRHPhwboLWHYx", "token_type": "Bearer", "expires_in": 15551999}'

In [134]:
pd.DataFrame(make_call_df(req,5))

{u'rating': 4.0, u'review_count': 21, u'name': u'Halal Food', u'url': u'https://www.yelp.com/biz/halal-food-new-york-6?adjust_creative=nzgGbAJQ1s77ZRvTEz_TaQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=nzgGbAJQ1s77ZRvTEz_TaQ', u'price': u'$', u'coordinates': {u'latitude': 40.71455, u'longitude': -74.00714}, u'image_url': u'https://s3-media1.fl.yelpcdn.com/bphoto/niRbFNJT5snZNlN1dfBldw/o.jpg', u'categories': [{u'alias': u'foodstands', u'title': u'Food Stands'}, {u'alias': u'halal', u'title': u'Halal'}], u'id': u'halal-food-new-york-6', u'is_closed': False, u'location': {u'city': u'New York', u'country': u'US', u'address2': u'', u'address3': u'', u'state': u'NY', u'address1': u'', u'zip_code': u''}}
{u'rating': 5.0, u'review_count': 37, u'name': u"Famous Fat Dave's Food Tours", u'url': u'https://www.yelp.com/biz/famous-fat-daves-food-tours-new-york?adjust_creative=nzgGbAJQ1s77ZRvTEz_TaQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=nzgGbA

,id_str,lat,lon,name,num_review,price,rating
0,halal-food-new-york-6,40.714550,-74.007140,Halal Food,21,1,4.0
1,famous-fat-daves-food-tours-new-york,40.714550,-74.007140,Famous Fat Dave's Food Tours,37,3,5.0
2,king-david-tacos-new-york,40.714550,-74.007140,King David Tacos,1,1,5.0
3,the-cardinal-club-new-york,40.714550,-74.007140,The Cardinal Club,2,-1,5.0
4,snap-dog-new-york,40.714550,-74.007140,Snap Dog,1,-1,3.0
5,limo-star-ny-new-york-2,40.712784,-74.005941,Limo Star NY,9,-1,4.0
6,west-side-highway-bike-path-new-york,40.714550,-74.007140,West Side Highway Bike Path,27,-1,4.0
7,mr-softy-new-york-3,NaN,NaN,Mr. Softy,2,1,5.0
8,citylisten-audio-tours-new-york,40.714550,-74.007140,CityListen Audio Tours,6,-1,4.5
9,afropolitan-insights-nyc-new-york,40.714550,-74.007140,Afropolitan Insights NYC,1,-1,1.0
